In [20]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support
import joblib
from datetime import datetime, timedelta
from typing import Dict, List, Union, Tuple

In [21]:
class AlertPredictor:
    def __init__(self, model_type='xgboost'):
        """
        Initializes the AlertPredictor with the specified model type ('xgboost' or 'randomforest').
        """
        self.model_type = model_type
        self.models = {}
        self.scalers = {}
        self.alert_types = ['LOW', 'MEDIUM', 'HIGH', ] # 'SIGMA']
        self.features = ['ChlPrs',
                         # 'hour',
                         # 'day_of_week',
                         # 'month',
                         # 'is_weekend',
                         'rolling_mean', 'rolling_std'] + [f'time_since_{at}' for at in self.alert_types]

    def load_and_preprocess_data(self, folder):
        """
        Loads and preprocesses data from CSV files in the specified folder.
        """
        dfs = []
        for i in range(9, 16):
            file_name = f"HTOL-{i:02d}_alerts.csv"
            df = pd.read_csv(os.path.join(folder, file_name))
            df['machine_id'] = f'HTOL-{i:02d}'
            dfs.append(df)

        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df['Time'] = pd.to_datetime(combined_df['Time'])
        combined_df = combined_df.sort_values(['machine_id', 'Time'])

        return combined_df

    def engineer_features(self, df):
        """
        Engineers features from the preprocessed data.
        """
        df['hour'] = df['Time'].dt.hour
        df['day_of_week'] = df['Time'].dt.dayofweek
        df['month'] = df['Time'].dt.month
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

        # Calculate rolling statistics
        df['rolling_mean'] = df.groupby('machine_id')['ChlPrs'].rolling(window=24, min_periods=1).mean().reset_index(0, drop=True)
        df['rolling_std'] = df.groupby('machine_id')['ChlPrs'].rolling(window=24, min_periods=1).std().reset_index(0, drop=True)

        # Calculate time since last alert for each type
        for alert_type in self.alert_types:
            df[f'time_since_{alert_type}'] = df.groupby('machine_id').apply(
                lambda x: x['Time'] - x[x['ALERT'] == alert_type]['Time'].shift(1)).reset_index(level=0, drop=True)
            df[f'time_since_{alert_type}'] = df[f'time_since_{alert_type}'].dt.total_seconds() / 3600  # Convert to hours

        return df

    def prepare_data_for_classification(self, df, target_alert_type, prediction_window):
        """
        Prepares the data for training the classification model.
        """
        df['target'] = df.groupby('machine_id').apply(
            lambda x: (x['ALERT'] == target_alert_type).rolling(window=prediction_window).max().shift(-prediction_window + 1)).reset_index(level=0,
                                                                                                                                         drop=True)

        X = df[self.features]
        y = df['target'].fillna(0)  # Fill NaN with 0 (no alert)

        return X, y

    def train_and_evaluate_classifier(self, X, y, test_size=0.2):
        """
        Trains and evaluates the classification model.
        """
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        if self.model_type == 'xgboost':
            # XGBoost configuration for imbalanced classification
            model = XGBClassifier(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=6,
                min_child_weight=1,
                gamma=0,
                subsample=0.8,
                colsample_bytree=0.8,
                scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]),  # Handle class imbalance
                random_state=42,
                eval_metric='logloss',
                early_stopping_rounds=10,
            )
            model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=0)
        elif self.model_type == 'randomforest':
            model = RandomForestClassifier(n_estimators=100, random_state=42)
            model.fit(X_train_scaled, y_train)
        else:
            raise ValueError("Invalid model_type. Choose 'xgboost' or 'randomforest'.")

        y_pred = model.predict(X_test_scaled)
        print(classification_report(y_test, y_pred))

        return model, scaler

    def train(self, folder, prediction_window=7):
        """
        Trains the models for each alert type.
        """
        df = self.load_and_preprocess_data(folder)
        df = self.engineer_features(df)

        for alert_type in self.alert_types:
            print(f"\nTraining model for {alert_type} alerts:")
            X, y = self.prepare_data_for_classification(df, alert_type, prediction_window)
            model, scaler = self.train_and_evaluate_classifier(X, y)
            self.models[alert_type] = model
            self.scalers[alert_type] = scaler

    def predict(self, new_data):
        """
        Makes predictions on new data.
        """
        predictions = {}
        for alert_type in self.alert_types:
            X_new = new_data[self.features]
            X_new_scaled = self.scalers[alert_type].transform(X_new)
            alert_probability = self.models[alert_type].predict_proba(X_new_scaled)[0, 1]
            predictions[alert_type] = alert_probability
        return predictions

    def visualize_alerts(self, df, target_alert_type, prediction_window, probability_threshold=0.7):
        """
        Visualizes actual alerts and high-risk periods.
        """
        X = df[self.features]
        X_scaled = self.scalers[target_alert_type].transform(X)

        df['alert_probability'] = self.models[target_alert_type].predict_proba(X_scaled)[:, 1]
        df['high_risk'] = df['alert_probability'] > probability_threshold

        plt.figure(figsize=(20, 15))
        machines = df['machine_id'].unique()
        n_machines = len(machines)

        for i, machine_id in enumerate(machines):
            machine_df = df[df['machine_id'] == machine_id]

            # Plot actual alerts
            alerts = machine_df[machine_df['ALERT'] == target_alert_type]
            plt.scatter(alerts['Time'], [i - 0.2] * len(alerts), marker='o', s=100,
                        label=f'Actual {target_alert_type} Alert' if i == 0 else "")

            # Plot high-risk periods
            high_risk_periods = machine_df[machine_df['high_risk']]
            plt.scatter(high_risk_periods['Time'], [i + 0.2] * len(high_risk_periods), marker='x', s=100, label=f'High Risk Period ({target_alert_type})' if i == 0 else "")

            plt.text(df['Time'].min(), i, machine_id, va='center', ha='right', fontweight='bold')

        plt.yticks(range(n_machines), machines)
        plt.xlabel('Date')
        plt.ylabel('Machine ID')
        plt.title(f'Actual Alerts vs High Risk Periods for {target_alert_type} Alerts')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [22]:

class ProductionAlertPredictor:
    def __init__(self):
        """
        Initialize the production predictor that handles both XGBoost and Random Forest models.
        """
        self.models = {}
        self.scalers = {}
        self.alert_types = ['LOW', 'MEDIUM', 'HIGH']
        self.features = [
            'ChlPrs',
            # 'hour',
            # 'day_of_week',
            # 'month',
            # 'is_weekend',
            'rolling_mean',
            'rolling_std'
        ] + [f'time_since_{at}' for at in self.alert_types]

    def save_models(self, xgb_predictor: AlertPredictor, rf_predictor: AlertPredictor,
                    save_dir: str) -> None:
        """
        Save trained models and scalers to disk.

        Args:
            xgb_predictor: Trained XGBoost AlertPredictor instance
            rf_predictor: Trained Random Forest AlertPredictor instance
            save_dir: Directory to save the models
        """
        os.makedirs(save_dir, exist_ok=True)

        # Save metadata
        metadata = {
            'features': self.features,
            'alert_types': self.alert_types,
            'timestamp': datetime.now().isoformat(),
        }
        joblib.dump(metadata, os.path.join(save_dir, 'metadata.joblib'))

        # Save models and scalers
        for alert_type in self.alert_types:
            # Save XGBoost models and scalers
            joblib.dump(
                xgb_predictor.models[alert_type],
                os.path.join(save_dir, f'xgboost_{alert_type.lower()}_model.joblib')
            )
            joblib.dump(
                xgb_predictor.scalers[alert_type],
                os.path.join(save_dir, f'xgboost_{alert_type.lower()}_scaler.joblib')
            )

            # Save Random Forest models and scalers
            joblib.dump(
                rf_predictor.models[alert_type],
                os.path.join(save_dir, f'randomforest_{alert_type.lower()}_model.joblib')
            )
            joblib.dump(
                rf_predictor.scalers[alert_type],
                os.path.join(save_dir, f'randomforest_{alert_type.lower()}_scaler.joblib')
            )

    def load_models(self, load_dir: str) -> None:
        """
        Load saved models and scalers from disk.

        Args:
            load_dir: Directory containing the saved models
        """
        # Load metadata
        metadata = joblib.load(os.path.join(load_dir, 'metadata.joblib'))
        self.features = metadata['features']
        self.alert_types = metadata['alert_types']

        # Initialize nested dictionaries for models and scalers
        self.models = {'xgboost': {}, 'randomforest': {}}
        self.scalers = {'xgboost': {}, 'randomforest': {}}

        # Load models and scalers
        for alert_type in self.alert_types:
            # Load XGBoost
            self.models['xgboost'][alert_type] = joblib.load(
                os.path.join(load_dir, f'xgboost_{alert_type.lower()}_model.joblib')
            )
            self.scalers['xgboost'][alert_type] = joblib.load(
                os.path.join(load_dir, f'xgboost_{alert_type.lower()}_scaler.joblib')
            )

            # Load Random Forest
            self.models['randomforest'][alert_type] = joblib.load(
                os.path.join(load_dir, f'randomforest_{alert_type.lower()}_model.joblib')
            )
            self.scalers['randomforest'][alert_type] = joblib.load(
                os.path.join(load_dir, f'randomforest_{alert_type.lower()}_scaler.joblib')
            )

    def prepare_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Prepare features for prediction.

        Args:
            data: DataFrame containing at minimum 'Time', 'ChlPrs', and 'machine_id' columns

        Returns:
            DataFrame with engineered features
        """
        df = data.copy()

        # Time-based features
        df['Time'] = pd.to_datetime(df['Time'])
        df['hour'] = df['Time'].dt.hour
        df['day_of_week'] = df['Time'].dt.dayofweek
        df['month'] = df['Time'].dt.month
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

        # Rolling statistics
        df['rolling_mean'] = df.groupby('machine_id')['ChlPrs'].rolling(
            window=24, min_periods=1).mean().reset_index(0, drop=True)
        df['rolling_std'] = df.groupby('machine_id')['ChlPrs'].rolling(
            window=24, min_periods=1).std().reset_index(0, drop=True)

        # Time since last alert features
        for alert_type in self.alert_types:
            if 'ALERT' in df.columns:
                df[f'time_since_{alert_type}'] = df.groupby('machine_id').apply(
                    lambda x: x['Time'] - x[x['ALERT'] == alert_type]['Time'].shift(1)
                ).reset_index(level=0, drop=True)
                df[f'time_since_{alert_type}'] = df[f'time_since_{alert_type}'].dt.total_seconds() / 3600
            else:
                # For new data without alert history, use a large value
                df[f'time_since_{alert_type}'] = 168  # One week in hours

        return df[self.features]

    def predict(self, data: pd.DataFrame, model_type: str = 'xgboost') -> Dict[str, Dict[str, float]]:
        """
        Make predictions using the loaded models.

        Args:
            data: DataFrame containing required features
            model_type: 'xgboost' or 'randomforest'

        Returns:
            Dictionary containing predictions for each machine and alert type
        """
        if model_type not in ['xgboost', 'randomforest']:
            raise ValueError("model_type must be 'xgboost' or 'randomforest'")

        # Prepare features
        X = self.prepare_features(data)

        # Make predictions for each machine and alert type
        predictions = {}
        for machine_id in data['machine_id'].unique():
            machine_data = X[data['machine_id'] == machine_id]
            machine_predictions = {}

            for alert_type in self.alert_types:
                # Scale the features
                X_scaled = self.scalers[model_type][alert_type].transform(machine_data)

                # Get prediction probabilities
                probs = self.models[model_type][alert_type].predict_proba(X_scaled)[:, 1]

                # Store the average probability for this alert type
                machine_predictions[alert_type] = float(probs.mean())

            predictions[machine_id] = machine_predictions

        return predictions

def save_trained_models(xgb_predictor: AlertPredictor, rf_predictor: AlertPredictor,
                        save_dir: str) -> None:
    """
    Convenience function to save trained models.
    """
    production_predictor = ProductionAlertPredictor()
    production_predictor.save_models(xgb_predictor, rf_predictor, save_dir)
    print(f"Models saved successfully to {save_dir}")

def load_production_predictor(load_dir: str) -> ProductionAlertPredictor:
    """
    Convenience function to load saved models.
    """
    production_predictor = ProductionAlertPredictor()
    production_predictor.load_models(load_dir)
    return production_predictor

In [23]:

class EnsembleAlertPredictor:
    def __init__(self, base_predictor: ProductionAlertPredictor,
                 lookback_window: int = 168,  # 7 days in hours
                 prediction_window: int = 168,  # 7 days in hours
                 agreement_threshold: float = 0.7):
        """
        Initialize the ensemble predictor that requires agreement between XGBoost and Random Forest.
        """
        self.predictor = base_predictor
        self.lookback_window = lookback_window
        self.prediction_window = prediction_window
        self.agreement_threshold = agreement_threshold

    def prepare_time_series_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Prepare time series features from historical data.
        """
        df = data.copy()

        # Ensure numeric type for ChlPrs
        df['ChlPrs'] = pd.to_numeric(df['ChlPrs'], errors='coerce')

        # Convert Time to datetime if it's not already
        df['Time'] = pd.to_datetime(df['Time'])

        # Sort the data
        df = df.sort_values(['machine_id', 'Time'])

        # Initialize features dictionary
        feature_dict = {
            'ChlPrs': df['ChlPrs'].copy(),  # Create a copy to avoid SettingWithCopyWarning
            'machine_id': df['machine_id'].copy(),  # Create a copy to avoid SettingWithCopyWarning
            'Time': df['Time'].copy()  # Create a copy to avoid SettingWithCopyWarning
        }

        # Calculate rolling statistics
        for machine in df['machine_id'].unique():
            mask = df['machine_id'] == machine
            # Use .transform to ensure the output is the same length as the original
            feature_dict['rolling_mean'] = df.loc[mask, 'ChlPrs'].rolling(
                window=24, min_periods=1).mean().transform(lambda x: x.fillna(method='bfill')).values
            feature_dict['rolling_std'] = df.loc[mask, 'ChlPrs'].rolling(
                window=24, min_periods=1).std().transform(lambda x: x.fillna(method='bfill')).values

        # Handle time since last alert features
        if 'ALERT' in df.columns:
            for alert_type in self.predictor.alert_types:
                # Initialize as an array of NaNs with the same length as the DataFrame
                feature_dict[f'time_since_{alert_type}'] = np.full(len(df), np.nan)
                for machine in df['machine_id'].unique():
                    mask = df['machine_id'] == machine
                    machine_data = df[mask].copy()
                    alert_times = machine_data[machine_data['ALERT'] == alert_type]['Time']
                    if not alert_times.empty:
                        last_alert = alert_times.shift(1)
                        hours_since = (machine_data['Time'] - last_alert).dt.total_seconds() / 3600
                        # Assign the values to the array using boolean indexing
                        feature_dict[f'time_since_{alert_type}'][mask] = hours_since.values
        else:
            for alert_type in self.predictor.alert_types:
                feature_dict[f'time_since_{alert_type}'] = 168.0  # One week in hours

        # Create new dataframe with features
        result_df = pd.DataFrame(feature_dict)

        # Fill missing values
        numeric_columns = result_df.select_dtypes(include=[np.number]).columns
        result_df[numeric_columns] = result_df[numeric_columns].fillna(method='ffill').fillna(method='bfill')

        return result_df

    def predict_window(self, historical_data: pd.DataFrame) -> Dict[str, Dict[str, Dict[str, float]]]:
        """
        Make predictions using both models and require agreement.
        """
        # Prepare features
        features_df = self.prepare_time_series_features(historical_data)

        # Get predictions from both models
        xgb_predictions = self.predictor.predict(features_df, model_type='xgboost')
        rf_predictions = self.predictor.predict(features_df, model_type='randomforest')

        # Combine predictions where models agree
        ensemble_predictions = {}

        for machine_id in features_df['machine_id'].unique():
            machine_xgb = xgb_predictions[machine_id]
            machine_rf = rf_predictions[machine_id]

            ensemble_predictions[machine_id] = {
                'agreed_alerts': {},
                'probabilities': {
                    'xgboost': machine_xgb,
                    'randomforest': machine_rf
                }
            }

            # Check for agreement between models
            for alert_type in self.predictor.alert_types:
                xgb_prob = machine_xgb[alert_type]
                rf_prob = machine_rf[alert_type]

                # Models agree if both predict above threshold
                if (xgb_prob > self.agreement_threshold and
                        rf_prob > self.agreement_threshold):
                    ensemble_predictions[machine_id]['agreed_alerts'][alert_type] = {
                        'probability': (xgb_prob + rf_prob) / 2,
                        'confidence': min(xgb_prob, rf_prob) / max(xgb_prob, rf_prob)
                    }

        return ensemble_predictions

    def visualize_predictions(self, historical_data: pd.DataFrame,
                              predictions: Dict[str, Dict[str, Dict[str, float]]],
                              machine_id: str = None):
        """
        Visualize the predictions alongside historical data.
        """
        if machine_id is None:
            machine_id = list(predictions.keys())[0]

        machine_data = historical_data[historical_data['machine_id'] == machine_id].copy()
        machine_data['Time'] = pd.to_datetime(machine_data['Time'])

        # Create the visualization
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), height_ratios=[2, 1])

        # Plot historical pressure data
        ax1.plot(machine_data['Time'], machine_data['ChlPrs'],
                 label='Pressure', color='blue', alpha=0.6)
        ax1.set_title(f'Historical Data and Predictions for {machine_id}')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Pressure')
        ax1.grid(True)

        # Add rolling mean and std to the plot
        features = self.prepare_time_series_features(machine_data)
        ax1.plot(features['Time'], features['rolling_mean'],
                 label='Rolling Mean (24h)', color='green', alpha=0.5)
        ax1.fill_between(features['Time'],
                         features['rolling_mean'] - features['rolling_std'],
                         features['rolling_mean'] + features['rolling_std'],
                         color='green', alpha=0.2, label='±1 Std Dev')
        ax1.legend()

        # Plot model probabilities
        machine_preds = predictions[machine_id]
        last_time = machine_data['Time'].max()
        prediction_times = [last_time + timedelta(hours=i)
                            for i in range(self.prediction_window)]

        for alert_type in self.predictor.alert_types:
            xgb_prob = machine_preds['probabilities']['xgboost'][alert_type]
            rf_prob = machine_preds['probabilities']['randomforest'][alert_type]

            ax2.plot([prediction_times[0], prediction_times[-1]],
                     [xgb_prob, xgb_prob],
                     '--', label=f'XGB {alert_type}')
            ax2.plot([prediction_times[0], prediction_times[-1]],
                     [rf_prob, rf_prob],
                     ':', label=f'RF {alert_type}')

            # Highlight agreed alerts
            if alert_type in machine_preds['agreed_alerts']:
                alert_info = machine_preds['agreed_alerts'][alert_type]
                ax2.axhspan(self.agreement_threshold,
                            alert_info['probability'],
                            alpha=0.3,
                            color='red',
                            label=f'Agreed {alert_type} Alert')

        ax2.axhline(y=self.agreement_threshold, color='r', linestyle='-',
                    alpha=0.3, label='Agreement Threshold')
        ax2.set_ylim(0, 1)
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Alert Probability')
        ax2.grid(True)
        ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

        plt.tight_layout()
        plt.show()

def simulate_production_monitoring(predictor: EnsembleAlertPredictor,
                                  data: pd.DataFrame,
                                  start_date: str,
                                  end_date: str,
                                  step_hours: int = 24):
    """
    Simulate production monitoring by stepping through time windows.
    """
    data = data.copy()
    data['Time'] = pd.to_datetime(data['Time'])

    current_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    results = []

    while current_date <= end_date:
        # Get historical window
        historical_window = data[
            (data['Time'] >= current_date - timedelta(hours=predictor.lookback_window)) &
            (data['Time'] <= current_date)
        ].copy()

        if not historical_window.empty:
            # Make prediction
            predictions = predictor.predict_window(historical_window)

            # Store results
            for machine_id, machine_preds in predictions.items():
                if machine_preds['agreed_alerts']:
                    results.append({
                        'time': current_date,
                        'machine_id': machine_id,
                        'predictions': machine_preds
                    })

            # Visualize some predictions
            if len(results) > 0 and len(results) % 5 == 0:  # Visualize every 5th prediction
                predictor.visualize_predictions(historical_window, predictions)

        current_date += timedelta(hours=step_hours)

    return results

# ... (rest of the code remains the same)

In [24]:
folder = "../../../outlier_tolerance=5_grouping_time_window=200_anomaly_threshold=6_start_date=2022-01-01_end_date=2026-01-01"
output_dir = "production_models_solo"

# Train models (if not already trained and saved)
# xgb_predictor = AlertPredictor(model_type='xgboost')
# xgb_predictor.train(folder, prediction_window)

# rf_predictor = AlertPredictor(model_type='randomforest')
# rf_predictor.train(folder, prediction_window)

# Save the trained models (if not already saved)
# save_trained_models(xgb_predictor, rf_predictor, output_dir)

# Load the production predictor
predictor = load_production_predictor(output_dir)

# Create the ensemble predictor
ensemble_predictor = EnsembleAlertPredictor(
    base_predictor=predictor,
    lookback_window=168,  # 7 days of historical data
    prediction_window=168,  # 7 days prediction window
    agreement_threshold=0.7
)

# Load some historical data for testing
data_files = [f for f in os.listdir(folder) if f.endswith('_alerts.csv')]
dfs = []
for file in data_files:
    df = pd.read_csv(os.path.join(folder, file))
    machine_id = file.split('_')[0]
    df['machine_id'] = machine_id
    dfs.append(df)

historical_data = pd.concat(dfs, ignore_index=True)

# Simulate production monitoring
results = simulate_production_monitoring(
    predictor=ensemble_predictor,
    data=historical_data,
    start_date='2023-01-01',
    end_date='2025-01-31',
    step_hours=24
)

# Print results
for result in results:
    print(f"\nPredictions for {result['machine_id']} at {result['time']}:")
    if result['predictions']['agreed_alerts']:
        for alert_type, alert_info in result['predictions']['agreed_alerts'].items():
            print(f"{alert_type} Alert: Probability = {alert_info['probability']:.3f}, "
                  f"Confidence = {alert_info['confidence']:.3f}")
    else:
        print("No agreed alerts.")

/var/folders/d7/0np89js16x9b596pzk8m108c0000gn/T/ipykernel_42071/487203991.py:41: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  window=24, min_periods=1).mean().transform(lambda x: x.fillna(method='bfill')).values
/var/folders/d7/0np89js16x9b596pzk8m108c0000gn/T/ipykernel_42071/487203991.py:43: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  window=24, min_periods=1).std().transform(lambda x: x.fillna(method='bfill')).values


ValueError: All arrays must be of the same length